In [52]:
from google.colab import files
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
import string
import gensim
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [11]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2022-11-04 15:08:05--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-11-04 15:08:05--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-11-04 15:08:06--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [41]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [3]:
folklore = pd.read_csv('08-folklore_deluxe_version.csv')
evermore = pd.read_csv('09-evermore_deluxe_version.csv')
lover = pd.read_csv('07-lover.csv')
reputation = pd.read_csv('06-reputation.csv')
the1989 = pd.read_csv('05-1989_deluxe.csv')
red = pd.read_csv('04-red_deluxe_edition.csv')
speak = pd.read_csv('03-speak_now_deluxe_package.csv')
fearless = pd.read_csv('02-fearless_taylors_version.csv')
taylor = pd.read_csv('01-taylor_swift.csv')
midnights = pd.read_csv('10_midnights.csv')

In [20]:
df = pd.concat([folklore, evermore, lover,reputation,the1989,red,speak,fearless,taylor], axis=0)
df.head()

,album_name,track_title,track_n,lyric,line
0,folklore (deluxe version),the 1,1,"I'm doing good, I'm on some new shit",1
1,folklore (deluxe version),the 1,1,"Been saying ""yes"" instead of ""no""",2
2,folklore (deluxe version),the 1,1,"I thought I saw you at the bus stop, I didn't ...",3
3,folklore (deluxe version),the 1,1,I hit the ground running each night,4
4,folklore (deluxe version),the 1,1,I hit the Sunday matinée,5


In [21]:
df = df[['track_title', 'lyric']]

In [22]:
df = pd.concat([df, midnights], axis = 0)

In [10]:
df.tail()

,track_title,lyric
15,Paris,Your ex-friend's sister met someone at a club ...
16,High Infidelity,Lock broken\nSlur spoken\nWound open\nGame tok...
17,Glitch,We were supposed to be just friends\nYou don't...
18,"Would've, Could've, Should've","If you would've blinked, then I would've\nLook..."
19,Dear Reader,Dear reader\nIf it feels like a trap\nYou're a...


In [ ]:
# preprocessing - zmienić didn't na did not itd

In [12]:
import re

In [23]:
df['lyric'] = df['lyric'].apply(lambda x: re.sub(r"\d+", "", x))

In [24]:
df['lyric'] = df['lyric'].apply(lambda x: re.sub(r"[.]?-[.]?", " ", x))

In [25]:
df['lyric'] = df['lyric'].apply(lambda x: x.replace("\n", " "))

In [26]:

translate_table = dict((ord(char), None) for char in string.punctuation) 
df['lyric'] = df['lyric'].apply(lambda x: x.translate(translate_table))

In [28]:
df.tail()

,track_title,lyric
15,Paris,Your ex friends sister met someone at a club a...
16,High Infidelity,Lock broken Slur spoken Wound open Game token ...
17,Glitch,We were supposed to be just friends You dont l...
18,"Would've, Could've, Should've",If you wouldve blinked then I wouldve Looked a...
19,Dear Reader,Dear reader If it feels like a trap Youre alre...


In [29]:
df['lyric'] = df['lyric'].apply(lambda x: x.lower())

In [30]:
def get_pos(word):

  #Słownik mapujący symbole części mowy zwrócone przez tagger do ich ekwiwalentów akceptowalnych przez lemmatizer
    tag_map = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

  #Tagowanie części mowy i pobieranie jej symbolu
    tag = nltk.pos_tag([word])[0][1][0].upper()

  #Zwracanie zmapowanej wartości - w przypadku braku wartością domyślną jest rzeczownik
    return tag_map.get(tag, wordnet.NOUN)

In [31]:
lemmatizer = WordNetLemmatizer()

In [32]:
def lemmatize(text):
    return ' '.join(lemmatizer.lemmatize(word, get_pos(word)) for word in nltk.word_tokenize(text))

In [33]:
df['lyric'] = df['lyric'].apply(lambda x: x.replace("\u2005", " "))

In [34]:
df['lyric'] = df['lyric'].apply(lambda x: lemmatize(x))

In [35]:
all_lyrics = ' '.join(list(df['lyric']))

In [36]:
words = nltk.word_tokenize(all_lyrics)

words=[word.lower() for word in words]
vocab = set(words)

In [38]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(vocab)

In [39]:
def embedding_for_vocab(filepath, word_index,
                        embedding_dim):
    vocab_size = len(word_index) + 1
      
    # Adding again 1 because of reserved 0 index
    embedding_matrix_vocab = np.zeros((vocab_size,
                                       embedding_dim))
  
    with open(filepath, encoding="utf8") as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word]
                embedding_matrix_vocab[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]
  
    return embedding_matrix_vocab

In [42]:
# matrix for vocab: word_index
embedding_dim = 50
embedding_matrix_vocab = embedding_for_vocab(
    './glove.6B.50d.txt', tokenizer.word_index,
  embedding_dim)
  
print("Dense vector for first word is => ",
      embedding_matrix_vocab[1])

Dense vector for first word is =>  [ 0.45682001 -0.60628998  0.85184002 -0.58557999 -0.54939002  0.21787
  0.27851999  0.69506001 -0.24624     0.26122001  0.25349     0.42471999
 -0.32273     0.18626     0.31700999  1.19809997  0.72521001 -0.49340999
  0.68366998 -0.89897001  0.060898   -0.18691     0.3107      0.21246
  0.43193001 -1.10010004 -0.39925    -0.64179999  0.32036    -0.22346
  2.32660007  0.91371    -0.91684002 -0.47479999  0.24237999 -0.13437
  0.45967001 -0.17704     0.028216   -0.70074999 -0.55855    -0.61886001
  0.52354997  0.34226999  0.22245     0.46869999 -0.041167    0.79280001
  0.29798001 -0.078627  ]


In [44]:
nn = NearestNeighbors(n_neighbors=4, algorithm='ball_tree', metric='euclidean')

In [45]:
nn.fit(embedding_matrix_vocab)

NearestNeighbors(algorithm='ball_tree', metric='euclidean', n_neighbors=4)

In [46]:
results = nn.kneighbors(embedding_matrix_vocab[[3]])

In [49]:
most_suitable = list(tokenizer.word_index.keys())[list(tokenizer.word_index.values()).index(results[1][0][0])]

In [50]:
most_suitable

'lurch'

In [53]:
vectorizer = TfidfVectorizer(
    analyzer='word', 
    stop_words='english', 
    ngram_range = (1,1), 
    #max_df=0.5, 
    use_idf=True, 
    smooth_idf=True
    #max_features=1000
    )

In [ ]:
df.head()

,album_name,track_title,track_n,lyric,line
0,folklore (deluxe version),the 1,1,Im do good Im on some new shit,1
1,folklore (deluxe version),the 1,1,Been say yes instead of no,2
2,folklore (deluxe version),the 1,1,I thought I saw you at the bus stop I didnt th...,3
3,folklore (deluxe version),the 1,1,I hit the ground run each night,4
4,folklore (deluxe version),the 1,1,I hit the Sunday matinée,5


In [54]:
grouped = df[['track_title', 'lyric']].groupby('track_title').agg(list).reset_index()

In [55]:
grouped['lyric'] = grouped['lyric'].apply(lambda x: ' '.join(x))

In [57]:
grouped.tail()

,track_title,lyric
178,​r​ight where you left me,friend break up friend get married stranger ge...
179,​the lakes,be it romantic how all my elegy eulogize me im...
180,​the last great american dynasty,rebekah rode up on the afternoon train it be s...
181,​tolerate it,i sit and watch you reading with your head low...
182,​’tis the damn season,if i want to know who you be hang with while i...


In [58]:
tfidf_matrix  = vectorizer.fit_transform(grouped['lyric'].tolist() )

In [59]:
tfidf_features = sorted(vectorizer.get_feature_names())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [60]:
df_tfidf = pd.DataFrame(data = tfidf_matrix.toarray(),index = range(len(grouped)),columns = tfidf_features)
df_tfidf

,abigail,absent,absurd,abusive,accent,accident,accidental,account,accuse,ace,...,yesterday,yeugh,yogurt,york,youd,youll,young,youre,yourе,youve
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.082846,0.037468,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.046119,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.043059,0.020324,0.0,0.0
179,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
180,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
181,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.056988,0.0,0.0


In [61]:
result = pd.concat([grouped, df_tfidf], axis=1, join="inner")

In [62]:
result.head()

,track_title,lyric,abigail,absent,absurd,abusive,accent,accident,accidental,account,...,yesterday,yeugh,yogurt,york,youd,youll,young,youre,yourе,youve
0,...Ready for It?,knew he be a killer first time that i saw him ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.082846,0.037468,0.000000,0.0,0.0
1,22,it feel like a perfect night to dress up like ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
2,A Perfectly Good Heart,why would you wan na break a perfectly good he...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
3,A Place In This World,i dont know what i want so dont ask me cause i...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
4,Afterglow,i blew thing out of proportion now youre blue ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.046119,0.0,0.0


In [65]:
result.nlargest(3,'snow')

,track_title,lyric,abigail,absent,absurd,abusive,accent,accident,accidental,account,...,yesterday,yeugh,yogurt,york,youd,youll,young,youre,yourе,youve
98,Snow On The Beach,one night a few moon ago i saw fleck of what c...,0.0,0.0,0.0,0.045659,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000
61,It’s Nice to Have a Friend,ooh ooh school bell ring walk me home sidewalk...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.05564
117,The Best Day (Taylor’s Version),im five year old it get cold ive get my big co...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.102132,0.0,0.00000


In [66]:
metadata = pd.read_csv('spotify_taylorswift.csv')

In [67]:
metadata = metadata[['name', 'danceability', 'acousticness', 'energy', 'liveness', 'valence']]

In [68]:
metadata_mid = pd.read_csv('midnights-spotify.csv')

In [70]:
final_metadata = pd.concat([metadata, metadata_mid], axis=0)

In [71]:
final = pd.merge(
    final_metadata,
    result,
    how="inner",
    on=None,
    left_on="name",
    right_on="track_title",
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

In [72]:
final.head()

,name,danceability,acousticness,energy,liveness,valence,track_title,lyric,abigail,absent,...,yesterday,yeugh,yogurt,york,youd,youll,young,youre,yourе,youve
0,22,0.661,0.00215,0.729,0.0477,0.668,22,it feel like a perfect night to dress up like ...,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
1,A Perfectly Good Heart,0.483,0.00349,0.751,0.1280,0.268,A Perfectly Good Heart,why would you wan na break a perfectly good he...,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
2,Afterglow,0.756,0.13000,0.449,0.1140,0.399,Afterglow,i blew thing out of proportion now youre blue ...,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.046119,0.0,0.000000
3,All Too Well,0.602,0.03820,0.609,0.1280,0.337,All Too Well,i walk through the door with you the air be co...,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.099881
4,Anti-Hero,0.638,0.13300,0.634,0.1520,0.519,Anti-Hero,i have this thing where i get old but just nev...,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.039772,0.0,0.000000


In [73]:
features = ['danceability', 'acousticness', 'energy', 'liveness', 'valence']

In [74]:
example = {'romance': 1}

In [75]:
# matrix for vocab: word_index
embedding_dim = 50
embedding = embedding_for_vocab(
    './glove.6B.50d.txt', example,
  embedding_dim)
  
print("Dense vector for first word is => ",
      embedding[1])

Dense vector for first word is =>  [ 2.59200007e-01  1.12230003e+00 -1.59300005e+00 -1.23690002e-01
 -1.50409997e-01  1.35230005e+00 -4.44849998e-01 -4.33490008e-01
 -6.17089987e-01  1.09710002e+00 -3.75299990e-01  7.91920006e-01
 -2.11909994e-01  3.76489997e-01  1.05530000e+00 -4.50250000e-01
 -4.01450008e-01  7.70549998e-02 -1.03349999e-01  4.41700011e-01
  4.38129991e-01  9.72060025e-01  9.50599983e-02  9.50380027e-01
  9.78630006e-01 -8.12189996e-01 -1.25880003e+00 -1.58629999e-01
  7.95669973e-01  1.23329997e-01  1.52730000e+00 -2.26129994e-01
  1.10670000e-01 -7.54309967e-02 -2.84299999e-02  9.93089983e-04
 -1.69369996e+00 -5.15169978e-01 -6.11320019e-01 -8.41170013e-01
  6.70820028e-02  1.83060005e-01  1.44930005e-01 -2.29589999e-01
 -7.61620030e-02  9.62060019e-02  1.29089999e+00 -5.79970002e-01
 -3.14269990e-01 -3.65330011e-01]


In [76]:
resultt = nn.kneighbors(embedding[[1]])

In [77]:
all_results = list(resultt[1][0])

In [78]:
top_words = [list(tokenizer.word_index.keys())[list(tokenizer.word_index.values()).index(x)] for x in all_results]

In [79]:
new_features = features + top_words

In [80]:
new_features

['danceability',
 'acousticness',
 'energy',
 'liveness',
 'valence',
 'romance',
 'romantic',
 'tale',
 'mystery']

In [ ]:
final[new_features]

,danceability,acousticness,energy,liveness,valence,romantic,mystery,revolves,obsession
0,0.661,0.00215,0.729,0.0477,0.668,0.0,0.0,0.0,0.0
1,0.483,0.00349,0.751,0.1280,0.268,0.0,0.0,0.0,0.0
2,0.756,0.13000,0.449,0.1140,0.399,0.0,0.0,0.0,0.0
3,0.602,0.03820,0.609,0.1280,0.337,0.0,0.0,0.0,0.0
4,0.646,0.08850,0.794,0.2010,0.287,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
94,0.456,0.81400,0.265,0.0679,0.112,0.0,0.0,0.0,0.0
95,0.602,0.88800,0.494,0.0902,0.374,0.0,0.0,0.0,0.0
96,0.780,0.75000,0.361,0.1080,0.163,0.0,0.0,0.0,0.0
97,0.511,0.31200,0.478,0.1470,0.408,0.0,0.0,0.0,0.0


In [ ]:
# z tej reszty zrobic potem srednia, tak zeby to byl jeden feature
# knn do jakis dacneability itd a reszte do 1